In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [ ]:
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [ ]:
from ast import literal_eval
df = pd.read_csv('../data/full_corrected_data.csv')
df['channels']  = df['channels'].apply(literal_eval)
df['intensities'] = df['intensities'].apply(literal_eval)
df['masses'] = df['masses'].apply(literal_eval)

In [ ]:
df.dropna(inplace=True)
df = df[df['intensities'].apply(len)> 0].copy()
df.head()

,file_name,MassOverTime,MassOffset,StartFlightTime,SpecBinSize,channels,intensities,masses,adjusted_original_proportion_identified,avg_dist_frags_low,avg_dist_frags_high,adjusted_proportion_identified,proportion_identified,diff,prop_diff_in_low,calibration
0,0106301.cas,0.387384,-0.275004,0.0,1.248,"[2644.0367300000003, 3505.0183700000002, 4162....","[73874, 1234, 138, 610, 1216, 4159, 8958, 1084...","[1.0065519723918102, 2.015029094672708, 3.0191...",0.398438,0.001298,0.002255,0.515625,0.515625,0.000958,0.738174,0
1,0107316.cas,0.387113,-0.278302,0.0,1.248,"[2647.00072, 3508.9949100000003, 4164.59326000...","[49864, 1034, 168, 4696, 8247, 13992, 17903, 2...","[1.00101811517532, 2.0077555328930656, 3.00565...",0.129771,0.001537,0.002586,0.511450,0.511450,0.001049,0.682225,0
2,0110203.cas,0.379037,-0.271056,0.0,4.992,"[1973.87665, 2049.0706800000003, 2122.01224, 2...","[23352, 74717, 10387, 947, 12344, 9121, 249, 4...","[11.998071176139083, 13.003971096434277, 14.01...",0.388889,0.001640,0.001858,0.444444,0.444444,0.000218,0.133100,0
3,0110212.cas,0.379177,-0.269744,0.0,4.992,"[672.00298, 891.90543, 1970.94521, 2046.11295,...","[34398, 304, 2223, 3521, 5205, 6509, 99, 115, ...","[1.0045194511091773, 2.012140966655108, 11.978...",0.379630,0.001337,0.002206,0.592593,0.592593,0.000868,0.649178,0
4,0116511.cas,0.383360,-0.302184,0.0,1.248,"[2726.98153, 3594.53649, 4265.28736, 7866.5038...","[42995, 602, 151, 17912, 9536, 17609, 29604, 5...","[1.0049940659400325, 2.0094784706009245, 3.022...",0.350427,0.001397,0.002171,0.487179,0.467213,0.000774,0.554114,0


In [ ]:
from data_transformation import get_frags
from data_transformation import mass_formula
frags = get_frags()['FragmentMass']

In [ ]:
from data_transformation import generate_calibrated_data, add_error, generate_data
placeholder = df[['file_name', 'MassOverTime', 'MassOffset', 'StartFlightTime',
       'SpecBinSize', 'channels', 'intensities', 'masses']].copy()
erred = generate_data(placeholder, 4, 2, [0, 1, 0], multi_class=False, sub_one=False,
                      slope_mods=True, slope_cat=True, slope_modifiers=[5, 2])
for num in range(1):
    for _ in range(1):
        erred = pd.concat([erred, generate_data(placeholder, 3.5, 1,
                                                [0.2, .6, 1], multi_class=False,
                                                sub_one=False, slope_cat=True)], axis=0)
erred.reset_index(inplace=True, drop=True)
erred = generate_calibrated_data(erred)

In [ ]:
def recalibrate(row, spots, slope_val, offset_val, augment=True,
                num_peaks=False):
    '''
    Function which quickly asseses the calibration of a spectrum.
    If augment is True slope_val and offset_val are treated as proportions
    of slope/offset to augment original values with. Otherwise the values
    are treated a the new values for slope and offset.

    Arguments -------
    row: row corresponding to spectra
    spots: pd.Series of fragments
    slope_val: either a proportion of slope to augment slope with or a new value
    for slope.
    offset_val: either a proportion of offset to augment offset with or a new
    value for offset.
    augment: whether to treat slope_val/offset_val as a proportion or value
    num_peaks: if true also returns the number of peaks of matched per spectrum
    '''
    if augment:
        slope = row.MassOverTime + slope_val * row.MassOverTime
        offset = row.MassOffset + offset_val * row.MassOffset
    else:
        slope = slope_val
        offset = offset_val
    
    peaks = []
    peaks = mass_formula(np.array(row.channels), row.SpecBinSize, row.StartFlightTime, slope,
                   offset)
    masses, frags1, dist1, masses2, frags2, dist2 = get_frags_dists(peaks,
                                                                    spots,
                                                                    [.003,.007])
    prop = len(masses) / (len(peaks[peaks<236]) + 1)
    low_dist = 0
    high_dist = 0
    if len(dist1) > 0:
        low_dist = np.mean(dist1)
    if len(dist2) > 0:
        high_dist = np.mean(dist2)
    if num_peaks:
        return prop, low_dist, high_dist, len(masses)
    else:
        return prop, low_dist, high_dist

In [ ]:
from data_transformation import get_frags_dists
def get_cols(data, frags, err=False):
    n = 1
    if err:
        n = 3
    cols = [[[] for num in range(n)] for num in range(4)]
    for i, row in enumerate(data.itertuples()):
        a, b, c, d = recalibrate(row, frags, 0, 0, num_peaks=True)
        cols[0][0].append(a)
        cols[1][0].append(b)
        cols[2][0].append(c)
        cols[3][0].append(d)
        if err:
            for i, num in enumerate([1, -1]):
                chan = get_amu_channel(row.SpecBinSize,
                                              row.StartFlightTime,
                                              row.MassOverTime,
                                              row.MassOffset)
                one_pt_five = get_amu_offset(row.SpecBinSize,
                                             row.StartFlightTime,
                                             row.MassOverTime, chan,
                                             100 + num * .0015)
                a, b, c, d = recalibrate(row, frags, row.MassOverTime,
                                         one_pt_five,
                                         augment=False,
                                         num_peaks=True)
                cols[0][i + 1].append(a)
                cols[1][i + 1].append(b)
                cols[2][i + 1].append(c)
                cols[3][i + 1].append(d)
    return cols

In [ ]:
cols = get_cols(erred, frags,  err=True)

In [ ]:
from data_transformation import (get_isotope_data, get_ranges,
                                 get_suspicious_peaks, get_peak_suspiciousness)
isotopes = get_isotope_data()
ranges = get_ranges(isotopes['Isotope Masses'], 2000)
suspicious_peaks = erred['masses'].apply(get_suspicious_peaks, args=(ranges, ))
num_sus_peaks = pd.Series(suspicious_peaks).apply(len)
prop_sus_peaks = num_sus_peaks / erred['masses'].apply(len)
peak_susses =  erred['masses'].apply(get_peak_suspiciousness, args=(ranges, ))
avg_peak_sussness = pd.Series(peak_susses).apply(np.mean)

In [ ]:
from sklearn.model_selection import train_test_split
d = { 'Miscalibrated': erred['target'],'Proportion Assigned': cols[0][0],
     'Low Distance From Fragment': cols[1][0],
     'High Distance From Fragment': cols[2][0],
     'Peaks Matched' : cols[3][0],
     'Diff Proportion w/ Offset Added': -1 * (np.array(cols[0][0]) - np.array(cols[0][1])),
     'Low Distance w/ Offset Added':   (np.array(cols[1][0]) - np.array(cols[1][1])),
     'High Distance w/ Offset Added':  (np.array(cols[2][0]) - np.array(cols[2][1])),
     'Peaks Matched Offset Added' :  -1 *( np.array(cols[3][0]) - np.array(cols[3][1])),
     'Proportion w/ Offset Subbed':   -1 * (np.array(cols[0][0]) - np.array(cols[0][2])),
     'Low Distance w/ Offset Subbed':  (np.array(cols[1][0]) - np.array(cols[1][2])),
     'High Distance w/ Offset Subbed':  (np.array(cols[2][0]) - np.array(cols[2][2])),
     'Peaks Matched Offset Subbed' :  -1 * (np.array(cols[3][0]) - np.array(cols[3][2])),
     'Number of NPZ Peaks': num_sus_peaks, 
     'Proportion of NPZ Peaks': prop_sus_peaks,
     'Average Distance into NPZ': avg_peak_sussness}

training_data = pd.DataFrame(d)

In [ ]:
#training_data = pd.read_csv('../data/training_datawbinaryclass.0005.csv')
#training_data.drop(['Unnamed: 0'], axis=1, inplace=True)
#training_data['Miscalibrated'] = training_data['Miscalibrated'].apply(lambda x: 0 if x == 0 else 1)
#training_data.head()

In [ ]:
corr_df = training_data.copy()
corr_df = pd.concat([corr_df], axis=1)
#corr_df = pd.concat([corr_df, pd.get_dummies(corr_df['Miscalibrated'])], axis=1)

In [ ]:
corr_df.columns = ['Miscalibrated', 'Proportion Assigned', 'Low Distance From Fragment',
       'High Distance From Fragment', 'Peaks Matched',
       'Diff Proportion w/ Offset Added', 'Low Distance w/ Offset Added',
       'High Distance w/ Offset Added', 'Peaks Matched Offset Added',
       'Proportion w/ Offset Subbed', 'Low Distance w/ Offset Subbed',
       'High Distance w/ Offset Subbed', 'Peaks Matched Offset Subbed',
       'Number of NPZ Peaks',
       'Proportion of NPZ Peaks', 'Average Distance into NPZ']
       #, 'No Error','Offset Error',
        #    'Slope Error']

In [ ]:
training_data['Miscalibrated'] = training_data['Miscalibrated'].apply(lambda x: 0 if x < 1 else 1)

In [1]:
corr_df = training_data.copy()
corr_df = pd.concat([corr_df], axis=1)

NameError: ignored

In [2]:
corr = corr_df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(15, 15))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, cmap='icefire_r',mask=mask, vmax=.3, center=0,
            square=True, linewidths=.5, annot=True, ax=ax, cbar_kws={"shrink": 0.5})
corr_df=None

NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X = training_data.drop(['Miscalibrated'], axis=1)
#y = erred['target']
y = training_data['Miscalibrated']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(erred[erred['target'] == 1])

666

In [ ]:
class_weights = {0 : 1 / (np.sum(training_data['Miscalibrated'] == 0) / len(training_data)),
                 1 : 1 / (np.sum(training_data['Miscalibrated'] == 1) / len(training_data)), 
                 2 : 1 / (np.sum(training_data['Miscalibrated'] == 2) / len(training_data))}
class_weights

{0: 2.439824945295405, 1: 2.6400947119179166, 2: 4.731258840169731}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(class_weight=class_weights)
parameters = {'criterion':('gini', 'entropy'), 'n_estimators':[75, 150], 'max_depth':[3, 31]}
clf_rfc = GridSearchCV(rfc, parameters)
clf_rfc.fit(X_train, y_train)
print('Test Score: ' + str(clf_rfc.score(X_test, y_test)))
print('Train Score: ' + str(clf_rfc.score(X_train, y_train)))

Test Score: 0.647982062780269
Train Score: 1.0


In [ ]:
erred.loc[4441]

file_name             Rock Mosaic Pos-085_MassShiftCorrected~pb1.cas
MassOverTime                                                0.383057
MassOffset                                                 -0.690761
StartFlightTime                                                    0
SpecBinSize                                                    0.138
channels           [32054.51903, 39922.38414, 78607.05751, 81303....
intensities        [26669.0, 371.0, 1547.0, 2419.0, 3522.0, 7626....
masses             [1.0074135373637747, 2.015291707169683, 12.003...
target                                                             2
err_prop_slope                                           0.000157553
err_prop_offset                                         -5.25867e-05
err_at_100amu                                             -0.0344169
Name: 4441, dtype: object

In [ ]:
preds_rfc = clf_rfc.predict(X_test)
wrong_rfc = X_test.loc[y_test!= preds_rfc].copy()
wrong_rfc['target'] = y_test[y_test!= preds_rfc]
wrong_rfc['Predicted'] = preds_rfc[y_test!= preds_rfc]
wrong_rfc.head()

,Proportion Assigned,Low Distance From Fragment,High Distance From Fragment,Peaks Matched,Diff Proportion w/ Offset Added,Low Distance w/ Offset Added,High Distance w/ Offset Added,Peaks Matched Offset Added,Proportion w/ Offset Subbed,Low Distance w/ Offset Subbed,High Distance w/ Offset Subbed,Peaks Matched Offset Subbed,Number of NPZ Peaks,Proportion of NPZ Peaks,Average Distance into NPZ,target,Predicted
247,0.636986,0.001417,0.002204,186,0.061644,0.000466,0.000472,18,-0.178082,-0.000334,-0.000645,-52,3,0.010204,0.006302,1,0
6083,0.226866,0.001330,0.002724,76,-0.008955,-0.000057,0.000022,-3,0.008955,-0.000003,0.000033,3,76,0.224852,0.060945,1,2
4727,0.420168,0.001570,0.002891,50,0.058824,0.000012,0.000157,7,0.042017,0.000035,0.000251,5,3,0.024194,0.007537,1,2
5241,0.229299,0.001303,0.002152,108,0.006369,0.000125,0.000064,3,-0.014862,-0.000113,-0.000195,-7,618,0.244849,0.110245,2,1
3263,0.813953,0.001206,0.001619,35,-0.046512,-0.000593,-0.000595,-2,0.023256,0.000412,0.000329,1,0,0.000000,0.000414,0,1


In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(boosting_type ='dart', class_weight=class_weights)
parameters = {'learning_rate':[0.01, 0.2],
              'n_estimators':[90, 110]}
clf_lgbm = GridSearchCV(lgbm, parameters)
clf_lgbm.fit(X_train, y_train)
print('Test Score: ' + str(clf_lgbm.score(X_test, y_test)))
print('Train Score: ' + str(clf_lgbm.score(X_train, y_train)))

Test Score: 0.6517189835575485
Train Score: 0.8600523168908819


In [ ]:
preds_lgbm = clf_lgbm.predict(X_test)
wrong_lgbm = X_test.loc[y_test!= preds_lgbm].copy()
wrong_lgbm['target'] = y_test[y_test!= preds_lgbm]
wrong_lgbm['Predicted'] = preds_lgbm[y_test!= preds_lgbm]
wrong_lgbm.head()

,Proportion Assigned,Low Distance From Fragment,High Distance From Fragment,Peaks Matched,Diff Proportion w/ Offset Added,Low Distance w/ Offset Added,High Distance w/ Offset Added,Peaks Matched Offset Added,Proportion w/ Offset Subbed,Low Distance w/ Offset Subbed,High Distance w/ Offset Subbed,Peaks Matched Offset Subbed,Number of NPZ Peaks,Proportion of NPZ Peaks,Average Distance into NPZ,target,Predicted
247,0.636986,0.001417,0.002204,186,0.061644,0.000466,0.000472,18,-0.178082,-0.000334,-0.000645,-52,3,0.010204,0.006302,1,0
6083,0.226866,0.001330,0.002724,76,-0.008955,-0.000057,0.000022,-3,0.008955,-0.000003,0.000033,3,76,0.224852,0.060945,1,2
4727,0.420168,0.001570,0.002891,50,0.058824,0.000012,0.000157,7,0.042017,0.000035,0.000251,5,3,0.024194,0.007537,1,2
3263,0.813953,0.001206,0.001619,35,-0.046512,-0.000593,-0.000595,-2,0.023256,0.000412,0.000329,1,0,0.000000,0.000414,0,1
544,0.393162,0.001285,0.002185,138,-0.011396,0.000041,-0.000049,-4,0.002849,0.000033,0.000023,1,139,0.190934,0.070543,0,1


In [ ]:
len(wrong_lgbm)

In [ ]:
wrong_lgbm[wrong_lgbm['Predicted']==0]

In [ ]:
wrong_lgbm[wrong_lgbm['target']==0]

In [ ]:
len(wrong_lgbm)